In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
# test case 0
processes = [ "A", "B", "C", "D"]
arrival_time = [0, 0, 0, 0]
burst_time = [5, 4, 2, 1]
waiting_time = [7, 7, 4, 6]
turnaround_time = [12, 11, 6, 7]
complete_time = [12, 11, 6, 7]
quanta = 2

In [3]:
# test case 1
processes = ['P0', 'P1', 'P2','P3','P4']
arrival_time = [0,1,2,3,7]
burst_time = [3,2,1,1,1]
waiting_time = [3,1,2,3,0]
turnaround_time = [6,3,3,4,1]
complete_time = [6,4,5,7,8]
quanta = 2

In [4]:
# Test case 2
processes = ['P0', 'P1', 'P2','P3']
arrival_time = [0,1,2,4]
burst_time = [10,5,8,2]
waiting_time = [15,11,13,4]
turnaround_time = [25,16,21,6]
complete_time = [25,17,23,10]
quanta = 2

In [5]:
# test case 3 
processes = ['A', 'B', 'C','D','E']
arrival_time = [0,0,2,3,1]
burst_time = [3,2,1,4,3]
waiting_time = [0,3,6,6,4]
turnaround_time = [3,5,7,10,7]
complete_time = [3,5,9,13,8]
quanta = 3

In [6]:
# test case 4
processes = ['P1','P2','P3','P4','P5','P6']
arrival_time = [0,25,30,60,100,105]
burst_time = [20,25,25,15,10,10]
waiting_time = [0,20,30,15,0,5]
turnaround_time = [20,45,55,30,10,15]
complete_time = [20,70,85,90,110,120]
quanta = 10

In [7]:
# test case 5
processes = ['P1','P2','P3','P4','P5','P6']
arrival_time = [0,1,2,3,4,5]
burst_time = [8,4,2,1,3,2]
waiting_time = [10,3,6,7,7,13]
turnaround_time = [18,7,8,8,10,15]
complete_time = [18,8,10,11,14,20]
quanta = 4

In [8]:
# test case 6 - to be checked
processes = ["A", "B", "C", "D"]
arrival_time = [0,1,2,3]
burst_time = [10,5,8,2]
waiting_time = [10,3,6,7]
turnaround_time = [18,7,8,8]
complete_time = [18,8,10,11]
quanta = 2

In [9]:
expected_answer = pd.DataFrame({'Process':processes, 'Arrival Time':arrival_time, 'Burst Time':burst_time, 
                               'Waiting Time':waiting_time, 'Turn Around Time':turnaround_time, 'Completion Time':complete_time})
expected_answer.sort_values(by=['Arrival Time', 'Burst Time'], ignore_index=True, inplace=True)

In [10]:
# generate a sorted timeline for the round robin algorithm

def sort_process_rr(processes, arrival_time, burst_time, quanta):
    end_time = sum(burst_time)
    current_time = 0
    rem_time = burst_time.copy()
    processes_list = [ list(tup) for tup in list(zip(arrival_time, burst_time, processes, rem_time)) ]
    processes_list.sort(key=lambda x: x[0])
    sorted_process_list = []
    queue = []
    
    while current_time < end_time or processes_list or queue:
        available_processes_list = [process for process in processes_list if process[0] <= current_time]
        
        for process in available_processes_list:
            if process not in queue:
                queue.append(process)
                processes_list.remove(process)
        
        if queue:
            process_sel = queue.pop(0)
            print("Process selected", process_sel)
            
            if process_sel[3] > quanta:
                process_sel[3] -= quanta
                
                for i in range(quanta):
                    current_time += 1
                    available_processes_list = [process for process in processes_list if process[0] <= current_time]
                    for process in available_processes_list:
                        if process not in queue:
                            queue.append(process)   
                            processes_list.remove(process)
             
                queue.append(process_sel)
                sorted_process_list.append([current_time - quanta, quanta, process_sel[2]])
                
            else:
                
                for i in range(process_sel[3]):
                    current_time += 1
                    available_processes_list = [process for process in processes_list if process[0] <= current_time]
                    for process in available_processes_list:
                        if process not in queue:
                            queue.append(process)
                            processes_list.remove(process)
                
                sorted_process_list.append([current_time - process_sel[3], process_sel[3], process_sel[2]])
                
            
        else:
            process_sel = processes_list.pop(0)
            print("Process selected", process_sel)
            current_time = process_sel[0]
            
            if process_sel[3] > quanta:
                process_sel[3] -= quanta
                for i in range(quanta):
                    current_time += 1
                    available_processes_list = [process for process in processes_list if process[0] <= current_time]
                    for process in available_processes_list:
                        if process not in queue:
                            queue.append(process)   
                            processes_list.remove(process)
                
                queue.append(process_sel)
                sorted_process_list.append([current_time - quanta , quanta, process_sel[2]])
            else:
                for i in range(process_sel[3]):
                    current_time += 1
                    available_processes_list = [process for process in processes_list if process[0] <= current_time]
                    for process in available_processes_list:
                        if process not in queue:
                            queue.append(process)
                            processes_list.remove(process)
                            
                sorted_process_list.append([current_time - process_sel[3], process_sel[3], process_sel[2]])
    
    print("\nSorted Processed List", sorted_process_list)
    return sorted_process_list

In [11]:
def generate_gantt_chart(timeline):
    end_at_prev = 0
    for i in timeline:
        start_at = i[0]
        end_at = i[1] + start_at
        
        if start_at != end_at_prev:
            print(f"{end_at_prev}-{start_at} : Idle | ", end="")
    
        print(f"{start_at}-{end_at} : Process {i[2]} | ", end="")
        
        end_at_prev = end_at

In [12]:
# writing a program for shortest job first scheduling algorithm with arrival times to return
# waiting time and turn around time for each process and average waiting time and average turn around time
# for all processes


def rr(process, arrival_time, burst_time, quanta):    
    # for preemptive algorithms
    
    timeline = sort_process_rr(processes, arrival_time, burst_time, quanta)
    processes_list = [ list(tup) for tup in list(zip( process, arrival_time, burst_time )) ]
    df = pd.DataFrame(processes_list, columns=['Process', 'Arrival Time', 'Burst Time' ])
    timeline_df = pd.DataFrame(timeline, columns=['Start Time', 'Duration', 'Process'])
    
    list_of_processes = df['Process'].tolist()
    for i in list_of_processes:
        temp = timeline_df.loc[timeline_df['Process'] == i, 'Start Time'] + timeline_df.loc[timeline_df['Process'] == i, 'Duration']
        df.loc[df['Process'] == i, 'Completion Time'] = temp.iloc[-1]

    df['Turn Around Time'] = df['Completion Time'] - df['Arrival Time']
    df['Waiting Time'] = df['Turn Around Time'] - df['Burst Time']    
    
    df = df[['Process', 'Arrival Time', 'Burst Time', 'Waiting Time', 'Turn Around Time', 'Completion Time']]
        
    print("\nProcess Details:")
    for i in range(len(df)):
        print(f"Process: {df['Process'].iloc[i]}, Wait Time: {df['Waiting Time'].iloc[i]}, Turn Around Time: {df['Turn Around Time'].iloc[i]}, Completion Time: {df['Completion Time'].iloc[i]}")
        
    print(f"\nAverage Waiting Time: {df['Waiting Time'].mean():.2f} ")
    print(f"Average Turn Around Time:  {df['Turn Around Time'].mean():.2f} ")
    return df.reset_index(drop=True) , timeline

In [13]:
actual_answer, timeline = rr(processes, arrival_time, burst_time, quanta)

Process selected [0, 10, 'A', 10]
Process selected [1, 5, 'B', 5]
Process selected [2, 8, 'C', 8]
Process selected [0, 10, 'A', 8]
Process selected [3, 2, 'D', 2]
Process selected [1, 5, 'B', 3]
Process selected [2, 8, 'C', 6]
Process selected [0, 10, 'A', 6]
Process selected [1, 5, 'B', 1]
Process selected [2, 8, 'C', 4]
Process selected [0, 10, 'A', 4]
Process selected [2, 8, 'C', 2]
Process selected [0, 10, 'A', 2]

Sorted Processed List [[0, 2, 'A'], [2, 2, 'B'], [4, 2, 'C'], [6, 2, 'A'], [8, 2, 'D'], [10, 2, 'B'], [12, 2, 'C'], [14, 2, 'A'], [16, 1, 'B'], [17, 2, 'C'], [19, 2, 'A'], [21, 2, 'C'], [23, 2, 'A']]

Process Details:
Process: A, Wait Time: 15.0, Turn Around Time: 25.0, Completion Time: 25.0
Process: B, Wait Time: 11.0, Turn Around Time: 16.0, Completion Time: 17.0
Process: C, Wait Time: 13.0, Turn Around Time: 21.0, Completion Time: 23.0
Process: D, Wait Time: 5.0, Turn Around Time: 7.0, Completion Time: 10.0

Average Waiting Time: 11.00 
Average Turn Around Time:  17.2

In [14]:
print("Quanta:", quanta, end="\n\n")
generate_gantt_chart(timeline)

Quanta: 2

0-2 : Process A | 2-4 : Process B | 4-6 : Process C | 6-8 : Process A | 8-10 : Process D | 10-12 : Process B | 12-14 : Process C | 14-16 : Process A | 16-17 : Process B | 17-19 : Process C | 19-21 : Process A | 21-23 : Process C | 23-25 : Process A | 

In [15]:
actual_answer

,Process,Arrival Time,Burst Time,Waiting Time,Turn Around Time,Completion Time
0,A,0,10,15.0,25.0,25.0
1,B,1,5,11.0,16.0,17.0
2,C,2,8,13.0,21.0,23.0
3,D,3,2,5.0,7.0,10.0


In [16]:
expected_answer

,Process,Arrival Time,Burst Time,Waiting Time,Turn Around Time,Completion Time
0,A,0,10,10,18,18
1,B,1,5,3,7,8
2,C,2,8,6,8,10
3,D,3,2,7,8,11


In [17]:
check_actual_answer = actual_answer.sort_values(by=['Arrival Time', 'Burst Time'], ignore_index=True)

In [18]:
diff = check_actual_answer.compare(expected_answer)

if diff.empty:
    print("Your answer is correct")
else:
    print("Your answer is incorrect")
    
diff

Your answer is incorrect


Waiting Time       Turn Around Time       Completion Time      
          self other             self other            self other
0         15.0    10             25.0    18            25.0    18
1         11.0     3             16.0     7            17.0     8
2         13.0     6             21.0     8            23.0    10
3          5.0     7              7.0     8            10.0    11

In [19]:
# # insert this line in rr() function before uncommenting this cell and running it
# # df = df.astype({'Completion Time': 'int64', 'Turn Around Time': 'int64', 'Waiting Time': 'int64'})

# from pandas.testing import assert_frame_equal
# assert_frame_equal(check_actual_answer, expected_answer)